# Chapter 17 Simultaneity by Future

- concurrent.futures library
- asyncronous working object - Future
- asyncio package

During waiting responses from network, another work would be performed by simultaneity. It makes CPU cloack effectively.
Simultaenity scripts are always faster than sequence script

In [ ]:
# flags.py
import os
import time
import sys

import requests # it is not a standard library. According to the convention, it wrote after standard library with space

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
           'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = 'downloads/'

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)
        
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc =cc.lower())
    resp = requests.get(url)
    return resp.content

def show(txt):
    print(text, end = ' ')
    sys.stdout.flush()
    
def download_many(cc_list): # critical part different with simultaneity version
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
    return len(cc_list)

def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() = t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))
    
if __name__ == '__main__':
    main(download_many)

In [ ]:
# download by concurrent.futures
# flags_threadpool.py

from concurrent import futures
from flags import save_flag, get_flag, show, main

MAX_WORKERS = 20 # max threads using in ThreadPoolExecutor
 
def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor: # executor.__exit__() calls executor.shutdown(wait=True) -> blocked until all threads completed.
        res = executor.map(download_one, sorted(cc_list)) # download_one methods calls by various threads.
        
    return len(list(res))

if __name__ == '__main__':
    main(download_many)

concurrent.futures.Future and asyncio.Future
Used implementing postpond calculations whether completed or not.

Future -> do not make object. It is exclusively create in the simultaneity framework.

In [ ]:
# flags_threadpool_ac.py

def download_many(cc_list):
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers = 3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))
            
        results = []
        for future in futures.as_completed(to_do):
            res = future.result()
            msg= '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)
        
    return len(results)

if __name__ == '__main__':
    main(download_many)

In [6]:
# demo_executor_map.py

from time import sleep, strftime
from concurrent import futures

def display(*args):
    print(strftime('[%H:%M:%S]'), end = ' ')
    print(*args)
    
def loiter(n):
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done'
    display(msg.format('\t'*n, n))
    return n *10

def main():
    display('Script starting')
    executor = futures.ThreadPoolExecutor(max_workers=3)
    results = executor.map(loiter, range(5))
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))
        


In [7]:
main()

[22:52:37] Script starting
[22:52:37] loiter(0): doing nothing for 0s...
[22:52:37][22:52:37] 	loiter(1): doing nothing for 1s...
 loiter(0): done
[22:52:37] 		loiter(2): doing nothing for 2s...
[22:52:37] results: <generator object Executor.map.<locals>.result_iterator at 0x7f7c54339d60>
[22:52:37] [22:52:37] 			loiter(3): doing nothing for 3s...
Waiting for individual results:
[22:52:37] result 0: 0
[22:52:38] 	loiter(1): done
[22:52:38] 				loiter(4): doing nothing for 4s...[22:52:38] result 1: 10

[22:52:39] 		loiter(2): done
[22:52:39] result 2: 20
[22:52:40] 			loiter(3): done
[22:52:40] result 3: 30
[22:52:42] 				loiter(4): done
[22:52:42] result 4: 40
